In [2]:
import ollama

In [3]:
def call_model(prompt, model="deepseek-coder"):
    messages = [
        {"role": "user", "content": prompt}
    ]
    response = ollama.chat(model=model, messages=messages)
    return response["message"]["content"]

In [5]:
def load_code(filepath):
    """Reads the contents of a code file as a string."""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"❌ File not found: {filepath}")
        return ""
    except Exception as e:
        print(f"❌ Error reading file: {e}")
        return ""

In [12]:
def build_prompt(code, mode="default"):
    """Constructs a code review prompt with markdown-style sections."""
    
    if mode == "default":
        instructions = """
You are a code reviewer. Read the following code and respond in markdown with three sections:

## Summary
Briefly explain what this code does.

## Issues
List any problems, inefficiencies, or bad practices you notice.

## Suggestions
Recommend improvements, refactors, or best practices.

Respond directly below each section header in clear, concise language.
"""
    elif mode == "beginner_friendly":
        instructions = """
You are a friendly code reviewer helping a beginner programmer.

Respond in markdown with three sections:
## What the Code Does
## Things to Improve
## Suggestions and Tips

Be kind and explain anything that might be confusing for a new coder.
"""
    elif mode == "strict":
        instructions = """
You're a senior software engineer performing a detailed and critical code review.

Be blunt. Use markdown headers:

## Code Summary
## Technical Issues
## Required Changes

Focus on performance, logic errors, maintainability, and standards compliance.
"""
    else:
        raise ValueError(f"Unknown review mode: {mode}")

    prompt = f"{instructions.strip()}\n\n```python\n{code.strip()}\n```"
    return prompt


In [14]:
def user_input():
    """Collects user input for code review — filepath, mode, and save toggle."""
    filepath = input("Enter the path to the code file (e.g., sample_scripts/test1.py): ").strip()
    
    mode = input("Choose review mode [default / beginner_friendly / strict] (default): ").strip()
    if mode == "":
        mode = "default"
    
    save_response = input("Save review to markdown file? [y/N]: ").strip().lower()
    save = save_response == "y"
    
    return filepath, mode, save

In [13]:
def save_review(text, path):
    """Saves the review text to the specified markdown or text file."""
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"✅ Review saved to: {path}")
    except Exception as e:
        print(f"❌ Failed to save review: {e}")

In [17]:
def review_code(filepath, mode="default", save=False):
    """Runs the full review pipeline for a given code file."""
    print("🔍 Loading code...")
    code = load_code(filepath)
    if not code:
        return "❌ No code loaded. Exiting."
    
    print("🧠 Building prompt...")
    prompt = build_prompt(code, mode)
    
    print("🤖 Sending to DeepSeek-Coder...")
    review = call_model(prompt, model="deepseek-coder")
    
    if save:
        filename = os.path.basename(filepath).replace(".py", "").replace(".txt", "")
        output_path = f"reviews/{filename}_review.md"
        save_review(review, output_path)
    
    return review


In [ ]:
filepath, mode, save = user_input()
result = review_code(filepath, mode, save)
print("\n📝 Review Output:\n")
print(result)